## Árbol de regresión

EL presente modelo busca predecir el precio que tiene un servicio de taxi tomando en cuenta parametros como: distancia del viaje y tiempo del recorrido.

Se importan las librerias necesarias

In [16]:
import pandas as pd
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from joblib import dump


Se hace el entrenamiento con 3 datasets de recorridos de taxis ecologicos para tres meses consecutivos del año 2023, se concatenan y se usan para el entrenamiento

In [17]:
d0 = pd.read_parquet("C:/Users/Acer/Downloads/green_tripdata_2023-12_limpio.parquet")
df1 = pd.read_parquet("C:/Users/Acer/Downloads/green_tripdata_2023-11_limpio.parquet")
df2 = pd.read_parquet("C:/Users/Acer/Downloads/green_tripdata_2023-10_limpio.parquet")
df = pd.concat([d0, df1, df2])

In [18]:
df.shape

(183020, 12)

Se filtra el data set con los valores de la columna 'payment type' mayor que -1 ya que -1 representa a los valores nulos

In [19]:
df = df[df['payment_type'] > -1]
df.shape

(115890, 12)

In [20]:
df.head(1)

,start_trip,end_trip,pu_location_id,do_location_id,passenger_count,trip_distance,fare_amount,tip_amount,total_amount,payment_type,congestion_surcharge,type_of_taxi
0,2023-12-01 00:27:37,2023-12-01 00:42:48,74,243,1,4.8,22.6,5.02,30.12,1,0,1


In [21]:
df.sample(5)

,start_trip,end_trip,pu_location_id,do_location_id,passenger_count,trip_distance,fare_amount,tip_amount,total_amount,payment_type,congestion_surcharge,type_of_taxi
8233,2023-10-05 08:08:00,2023-10-05 08:16:31,74,43,1,1.37,10.0,2.30,13.80,1,0,1
5258,2023-12-03 16:59:07,2023-12-03 17:09:12,95,93,2,1.85,12.8,1.00,15.30,1,0,1
24853,2023-12-13 08:42:27,2023-12-13 08:51:05,74,166,1,1.29,10.7,3.66,15.86,1,0,1
5355,2023-10-03 19:46:29,2023-10-03 19:59:47,97,17,1,1.92,12.8,3.36,20.16,1,0,1
3055,2023-11-02 11:40:13,2023-11-02 12:13:06,116,231,1,9.00,41.5,0.00,45.75,1,2,1


Se extrate informacion de las fechas de inicio y fin de viaje para generar columnas con los siguientes valores: año, mes, dia y minutos de viaje

In [22]:
df['year'] = df['start_trip'].dt.year
df['month'] = df['start_trip'].dt.month
df['day'] = df['start_trip'].dt.dayofweek
df['min_trip'] = (df['end_trip'] - df['start_trip']).dt.total_seconds() / 60
df.head(1)

,start_trip,end_trip,pu_location_id,do_location_id,passenger_count,trip_distance,fare_amount,tip_amount,total_amount,payment_type,congestion_surcharge,type_of_taxi,year,month,day,min_trip
0,2023-12-01 00:27:37,2023-12-01 00:42:48,74,243,1,4.8,22.6,5.02,30.12,1,0,1,2023,12,4,15.183333


In [23]:
# Arbol de decision

In [24]:
# Selección de las variables independientes
X = df[['min_trip', 'trip_distance']]


# Selección de la variable dependiente
y = df['fare_amount']

# Separación del conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Creación del clasificador de árbol de decisión
arbol = tree.DecisionTreeRegressor(max_depth = 8, random_state=42, )

# Entrenamiento del modelo
arbol.fit(X_train, y_train)

# Evaluación del modelo
score = arbol.score(X_test, y_test)

# Predicción del recargo por congestión
predicciones = arbol.predict(X_test)


In [25]:
# Cálculo del error cuadrático medio (MSE)
mse = mean_squared_error(y_test, predicciones)

# Cálculo del R cuadrado (R^2)
r2 = r2_score(y_test, predicciones)

# Impresión de las métricas de evaluación
print("MSE:", mse)
print("R^2:", r2)

MSE: 18.68058473880605
R^2: 0.8582284535896589


In [26]:
resultados = pd.DataFrame({'Valor real': y_test, 'Valor predicho': predicciones}).head(5)
print(score)

# Impresión del dataframe
print(resultados.to_string())

0.8582284535896589
       Valor real  Valor predicho
30636       27.50       29.027773
42922       10.70       11.100245
52732        6.50        7.229406
5964        12.10       12.405741
47755       13.50       12.739585
14982       18.40       19.036012
974          9.30        9.671416
31508       95.00      100.020000
55413       17.70       18.630810
45425        8.60        8.494825
42709       45.70       48.296269
18335       12.80       12.448304
32583       17.00       16.958790
51608       20.00       22.830463
30592       12.80       12.739585
54771       12.10       12.739585
48579       11.40       11.100245
17001       33.80       32.267931
56277       10.00       10.256437
51569       17.70       17.655608
23828       16.30       16.958790
3831        13.50       13.247597
32108       18.40       18.164318
9475        35.90       34.994729
22137       31.00       28.192952
54692       15.00        7.817647
44069        7.90        9.254898
10042       10.00        9.67

Se observa un Score de 0.85, y se visualiza una columna con valores reales y valores predichos

Se guarda el modelo en formato joblib, para ser posteriormente consumido por Streamlit

In [27]:
dump(arbol, "E:/proyecto_final/Deploy_modelo/prediccion_precio.joblib")

['E:/proyecto_final/Deploy_modelo/prediccion_precio.joblib']